# <center> Kaggle Competition Assignment <center> Ian Brandenburg (2304791) <center> [GitHub Repo](https://github.com/Iandrewburg/Data_Science/tree/main/Data_Science_2/Assignments/Take_Home_Final)
    
    
The Kaggle competition has been launched, please register using this [link](https://www.kaggle.com/t/f79b637ede074e70a233661b4614083c).

You will find the training and test data in the data section of the competition, along with a description of the features. You will need to build models on the training data and make predictions on the test data and submit your solutions to Kaggle. You will also find a sample solution file in the data section that shows the format you will need to use for your own submissions.

The deadline for Kaggle solutions is 8PM on 19 April. You will be graded primarily on the basis of your work and how clearly you explain your methods and results. Those in the top three in the competition will receive some extra points. I expect you to experiment with all the methods we have covered: linear models, random forest, gradient boosting, neural networks + parameter tuning, feature engineering.

You will see the public score of your best model on the leaderboard. A private dataset will be used to evaluate the final performance of your model to avoid overfitting based on the leaderboard.

You should also submit to Moodle the documentation (ipynb and pdf) of your work, including exploratory data analysis, data cleaning, parameter tuning and evaluation. Aim for concise explanations.

### Import Libraries
---

In [36]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from itertools import combinations


import warnings
warnings.filterwarnings('ignore')

# Data Wrangling
---

## Data Import
---

In [3]:
train_data = pd.read_csv("https://raw.githubusercontent.com/Iandrewburg/Data_Science/main/Data_Science_2/Assignments/Take_Home_Final/train.csv")
train_data.head()


,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,is_popular,article_id
0,594,9,702,0.454545,1.0,0.620438,11,2,1,0,...,1.000000,-0.153395,-0.4,-0.10,0.0,0.0,0.5,0.0,0,1
1,346,8,1197,0.470143,1.0,0.666209,21,6,2,13,...,1.000000,-0.308167,-1.0,-0.10,0.0,0.0,0.5,0.0,0,3
2,484,9,214,0.618090,1.0,0.748092,5,2,1,0,...,0.433333,-0.141667,-0.2,-0.05,0.0,0.0,0.5,0.0,0,5
3,639,8,249,0.621951,1.0,0.664740,16,5,8,0,...,0.500000,-0.500000,-0.8,-0.40,0.0,0.0,0.5,0.0,0,6
4,177,12,1219,0.397841,1.0,0.583578,21,1,1,2,...,0.800000,-0.441111,-1.0,-0.05,0.0,0.0,0.5,0.0,0,7


In [4]:
test_data = pd.read_csv("https://raw.githubusercontent.com/Iandrewburg/Data_Science/main/Data_Science_2/Assignments/Take_Home_Final/test.csv")
test_data.head()


,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,article_id
0,134,11,217,0.631579,1.0,0.818966,4,2,2,0,...,0.136364,0.5,-0.170370,-0.200000,-0.155556,0.288889,-0.155556,0.211111,0.155556,2
1,415,11,1041,0.489423,1.0,0.700321,22,3,0,14,...,0.050000,1.0,-0.426268,-1.000000,-0.100000,0.975000,0.300000,0.475000,0.300000,4
2,625,9,486,0.599585,1.0,0.727273,4,3,1,0,...,0.062500,0.7,-0.387821,-1.000000,-0.050000,0.000000,0.000000,0.500000,0.000000,10
3,148,14,505,0.509018,1.0,0.718861,8,4,1,1,...,0.100000,1.0,-0.284722,-0.400000,-0.050000,0.000000,0.000000,0.500000,0.000000,13
4,294,14,274,0.620301,1.0,0.726190,5,1,1,0,...,0.100000,0.6,-0.333333,-0.333333,-0.333333,0.000000,0.000000,0.500000,0.000000,26


In [34]:
test_data.columns

Index(['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'glob

## Exploratory Data Analysis
---

### Variable Descriptions
---


    timedelta: Days between the article publication and the dataset acquisition (non-predictive)
    n_tokens_title: Number of words in the title
    n_tokens_content: Number of words in the content
    n_unique_tokens: Rate of unique words in the content
    n_non_stop_words: Rate of non-stop words in the content
    n_non_stop_unique_tokens: Rate of unique non-stop words in the content
    num_hrefs: Number of links
    num_self_hrefs: Number of links to other articles published by Mashable
    num_imgs: Number of images
    num_videos: Number of videos
    average_token_length: Average length of the words in the content
    num_keywords: Number of keywords in the metadata
    data_channel_is_lifestyle: Is data channel 'Lifestyle'?
    data_channel_is_entertainment: Is data channel 'Entertainment'?
    data_channel_is_bus: Is data channel 'Business'?
    data_channel_is_socmed: Is data channel 'Social Media'?
    data_channel_is_tech: Is data channel 'Tech'?
    data_channel_is_world: Is data channel 'World'?
    kw_min_min: Worst keyword (min. shares)
    kw_max_min: Worst keyword (max. shares)
    kw_avg_min: Worst keyword (avg. shares)
    kw_min_max: Best keyword (min. shares)
    kw_max_max: Best keyword (max. shares)
    kw_avg_max: Best keyword (avg. shares)
    kw_min_avg: Avg. keyword (min. shares)
    kw_max_avg: Avg. keyword (max. shares)
    kw_avg_avg: Avg. keyword (avg. shares)
    self_reference_min_shares: Min. shares of referenced articles in Mashable
    self_reference_max_shares: Max. shares of referenced articles in Mashable
    self_reference_avg_sharess: Avg. shares of referenced articles in Mashable
    weekday_is_monday: Was the article published on a Monday?
    weekday_is_tuesday: Was the article published on a Tuesday?
    weekday_is_wednesday: Was the article published on a Wednesday?
    weekday_is_thursday: Was the article published on a Thursday?
    weekday_is_friday: Was the article published on a Friday?
    weekday_is_saturday: Was the article published on a Saturday?
    weekday_is_sunday: Was the article published on a Sunday?
    is_weekend: Was the article published on the weekend?
    LDA_00: Closeness to LDA topic 0
    LDA_01: Closeness to LDA topic 1
    LDA_02: Closeness to LDA topic 2
    LDA_03: Closeness to LDA topic 3
    LDA_04: Closeness to LDA topic 4
    global_subjectivity: Text subjectivity
    global_sentiment_polarity: Text sentiment polarity
    global_rate_positive_words: Rate of positive words in the content
    global_rate_negative_words: Rate of negative words in the content
    rate_positive_words: Rate of positive words among non-neutral tokens
    rate_negative_words: Rate of negative words among non-neutral tokens
    avg_positive_polarity: Avg. polarity of positive words
    min_positive_polarity: Min. polarity of positive words
    max_positive_polarity: Max. polarity of positive words
    avg_negative_polarity: Avg. polarity of negative words
    min_negative_polarity: Min. polarity of negative words
    max_negative_polarity: Max. polarity of negative words
    title_subjectivity: Title subjectivity
    title_sentiment_polarity: Title polarity
    abs_title_subjectivity: Absolute subjectivity level
    abs_title_sentiment_polarity: Absolute polarity level
    is_popular: Whether or not the article was among the most popular ones based on shares on social media
    article_id: Unique identifier of the article


In [25]:
train_data.describe()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,is_popular,article_id
count,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,...,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000,29733.000000
mean,355.645646,10.390812,545.008274,0.555076,1.005852,0.695432,10.912690,3.290788,4.524535,1.263546,...,0.757780,-0.259709,-0.520981,-0.107793,0.281878,0.069691,0.341427,0.155234,0.121649,19834.913530
std,214.288261,2.110135,469.358037,4.064572,6.039655,3.768796,11.316508,3.840874,8.213823,4.189080,...,0.247293,0.128488,0.290454,0.095672,0.323461,0.264379,0.188735,0.225066,0.326886,11432.376037
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,1.000000
25%,164.000000,9.000000,246.000000,0.471400,1.000000,0.626126,4.000000,1.000000,1.000000,0.000000,...,0.600000,-0.328704,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000,0.000000,9965.000000
50%,342.000000,10.000000,409.000000,0.539894,1.000000,0.690566,8.000000,2.000000,1.000000,0.000000,...,0.800000,-0.252827,-0.500000,-0.100000,0.144444,0.000000,0.500000,0.000000,0.000000,19859.000000
75%,545.000000,12.000000,712.000000,0.609375,1.000000,0.755208,14.000000,4.000000,4.000000,1.000000,...,1.000000,-0.186494,-0.300000,-0.050000,0.500000,0.136364,0.500000,0.250000,0.000000,29742.000000
max,731.000000,23.000000,8474.000000,701.000000,1042.000000,650.000000,304.000000,74.000000,111.000000,91.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,1.000000,39643.000000


In [24]:
print(f"The shape of the training set is {train_data.shape[0]} rows, and {train_data.shape[1]} columns.") 

The shape of the training set is 29733 rows, and 61 columns.


In [23]:
total_missing_values = train_data.isnull().sum()[train_data.isnull().sum() > 0].sum()
print(f"There are a total of {total_missing_values} missing values in the dataset.")


There are a total of 0 missing values in the dataset.


In [8]:
train_data.columns

Index(['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'glob

## Feature Engineering
---

In [26]:
# Defining variable groups
basic_text_features = ['n_tokens_title',
                       'n_tokens_content',
                       'n_unique_tokens',
                       'n_non_stop_words',
                       'n_non_stop_unique_tokens',
                       'average_token_length',
                       'num_keywords']
content_properties = ['num_hrefs',
                      'num_self_hrefs',
                      'num_imgs',
                      'num_videos',
                      'global_subjectivity',
                      'global_sentiment_polarity',
                      'global_rate_positive_words',
                      'global_rate_negative_words']
keyword_performance = ['kw_min_min',
                       'kw_max_min',
                       'kw_avg_min',
                       'kw_min_max',
                       'kw_max_max',
                       'kw_avg_max',
                       'kw_min_avg',
                       'kw_max_avg',
                       'kw_avg_avg']
self_reference_metrics = ['self_reference_min_shares',
                          'self_reference_max_shares',
                          'self_reference_avg_sharess']
publication_timing = ['weekday_is_monday',
                      'weekday_is_tuesday',
                      'weekday_is_wednesday',
                      'weekday_is_thursday',
                      'weekday_is_friday',
                      'weekday_is_saturday',
                      'weekday_is_sunday',
                      'is_weekend']
content_topic_and_sentiment = ['data_channel_is_lifestyle',
                               'data_channel_is_entertainment',
                               'data_channel_is_bus',
                               'data_channel_is_socmed',
                               'data_channel_is_tech',
                               'data_channel_is_world',
                               'LDA_00',
                               'LDA_01',
                               'LDA_02',
                               'LDA_03',
                               'LDA_04',
                               'rate_positive_words',
                               'rate_negative_words',
                               'avg_positive_polarity',
                               'min_positive_polarity', 
                               'max_positive_polarity',
                               'avg_negative_polarity',
                               'min_negative_polarity',
                               'max_negative_polarity']
title_sentiment = ['title_subjectivity',
                   'title_sentiment_polarity',
                   'abs_title_subjectivity',
                   'abs_title_sentiment_polarity']


In [29]:
##################SQUARED TERMS###################
# square basic features in the training set
for var in basic_text_features:
    train_data[f'{var}_squared'] = train_data[var] ** 2

# square basic features in the test set
for var in basic_text_features:
    test_data[f'{var}_squared'] = test_data[var] ** 2
    
# square title sentiment features in the training set
for var in title_sentiment:
    train_data[f'{var}_squared'] = train_data[var] ** 2

# square title sentiment features in the test set
for var in title_sentiment:
    test_data[f'{var}_squared'] = test_data[var] ** 2
    
################INTERACTION TERMS##################
# Interacting the basic features
for (var1, var2) in combinations(basic_text_features, 2):
    train_data[f'{var1}_{var2}_interaction'] = train_data[var1] * train_data[var2]
    
for (var1, var2) in combinations(basic_text_features, 2):
    test_data[f'{var1}_{var2}_interaction'] = test_data[var1] * test_data[var2]

# Interacting the title sentiment features
for (var1, var2) in combinations(title_sentiment, 2):
    train_data[f'{var1}_{var2}_interaction'] = train_data[var1] * train_data[var2]
    
for (var1, var2) in combinations(title_sentiment, 2):
    test_data[f'{var1}_{var2}_interaction'] = test_data[var1] * test_data[var2]

In [30]:
sqrd_title_sentiment = ['title_subjectivity_squared',
                        'title_sentiment_polarity_squared',
                        'abs_title_subjectivity_squared',
                        'abs_title_sentiment_polarity_squared']
sqrd_basic_text_features = ['n_tokens_title_squared',
                            'n_tokens_content_squared',
                            'n_unique_tokens_squared',
                            'n_non_stop_words_squared',
                            'n_non_stop_unique_tokens_squared',
                            'average_token_length_squared',
                            'num_keywords_squared']
interaction_basic_text_features = ['n_tokens_title_n_tokens_content_interaction',
                                  'n_tokens_title_n_unique_tokens_interaction',
                                  'n_tokens_title_n_non_stop_words_interaction',
                                  'n_tokens_title_n_non_stop_unique_tokens_interaction',
                                  'n_tokens_title_average_token_length_interaction',
                                  'n_tokens_title_num_keywords_interaction',
                                  'n_tokens_content_n_unique_tokens_interaction',
                                  'n_tokens_content_n_non_stop_words_interaction',
                                  'n_tokens_content_n_non_stop_unique_tokens_interaction',
                                  'n_tokens_content_average_token_length_interaction',
                                  'n_tokens_content_num_keywords_interaction',
                                  'n_unique_tokens_n_non_stop_words_interaction',
                                  'n_unique_tokens_n_non_stop_unique_tokens_interaction',
                                  'n_unique_tokens_average_token_length_interaction',
                                  'n_unique_tokens_num_keywords_interaction',
                                  'n_non_stop_words_n_non_stop_unique_tokens_interaction',
                                  'n_non_stop_words_average_token_length_interaction',
                                  'n_non_stop_words_num_keywords_interaction',
                                  'n_non_stop_unique_tokens_average_token_length_interaction',
                                  'n_non_stop_unique_tokens_num_keywords_interaction',
                                  'average_token_length_num_keywords_interaction']
interaction_title_sentiment_features = ['title_subjectivity_title_sentiment_polarity_interaction',
                                       'title_subjectivity_abs_title_subjectivity_interaction',
                                       'title_subjectivity_abs_title_sentiment_polarity_interaction',
                                       'title_sentiment_polarity_abs_title_subjectivity_interaction',
                                       'title_sentiment_polarity_abs_title_sentiment_polarity_interaction',
                                       'abs_title_subjectivity_abs_title_sentiment_polarity_interaction']


In [87]:
test_data.columns

Index(['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'glob

In [31]:
# Define models
models = {
    'M1': basic_text_features,
    'M2': basic_text_features + content_properties,
    'M3': basic_text_features + content_properties + keyword_performance,
    'M4': basic_text_features + content_properties + keyword_performance + self_reference_metrics,
    'M5': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing,
    'M6': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing + content_topic_and_sentiment,
    'M7': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing + content_topic_and_sentiment + title_sentiment, 
    'M8': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing + content_topic_and_sentiment + title_sentiment + sqrd_title_sentiment,
    'M9': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing + content_topic_and_sentiment + title_sentiment + sqrd_title_sentiment + sqrd_basic_text_features,
    'M10': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing + content_topic_and_sentiment + title_sentiment + sqrd_title_sentiment + sqrd_basic_text_features + interaction_basic_text_features,
    'M11': basic_text_features + content_properties + keyword_performance + self_reference_metrics + publication_timing + content_topic_and_sentiment + title_sentiment + sqrd_title_sentiment + sqrd_basic_text_features + interaction_basic_text_features + interaction_title_sentiment_features
}

In [59]:
# Split 'train_data' into training and validation sets
X = train_data.drop(['is_popular', 'timedelta', 'article_id'], axis=1)
y = train_data['is_popular']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=20240407)

# Iterate through each model, fit, and evaluate on the validation set
for model_name, features in models.items():
    # Select only the features for the current model increment
    X_train_subset = X_train[features]
    X_val_subset = X_val[features]

    # Create a pipeline with standard scaling and logistic regression
    pipeline = make_pipeline(StandardScaler(), LogisticRegression())

    # Fit the model on the training subset
    pipeline.fit(X_train_subset, y_train)

    # Predict on the validation subset and evaluate
    y_pred_val = pipeline.predict(X_val_subset)
    accuracy = accuracy_score(y_val, y_pred_val)

    print(f'{model_name} Validation Accuracy: {accuracy:.4f}')

M1 Validation Accuracy: 0.8734
M2 Validation Accuracy: 0.8735
M3 Validation Accuracy: 0.8732
M4 Validation Accuracy: 0.8724
M5 Validation Accuracy: 0.8724
M6 Validation Accuracy: 0.8732
M7 Validation Accuracy: 0.8730
M8 Validation Accuracy: 0.8730
M9 Validation Accuracy: 0.8730
M10 Validation Accuracy: 0.8739
M11 Validation Accuracy: 0.8737


# Models
---

## Logistic Regression
---

In [60]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
import pandas as pd

results = []

for model_name, features in models.items():
    # Append "Logistic Regression" to the model name for clarity
    full_model_name = f"{model_name} Logistic Regression"

    # Define steps for pipeline: feature scaling and logistic regression
    steps = [
        ("scale_features", ColumnTransformer([("scale", StandardScaler(), features)], remainder='drop')),
        ("log_reg", LogisticRegression())
    ]

    # Create pipeline
    pipeline = Pipeline(steps)

    # Fit the model on training data
    pipeline.fit(X_train[features], y_train)

    # Predict probabilities on the training and validation data
    # Note: We use predict_proba to get probabilities, and we're interested in the probabilities of the positive class (usually at index 1)
    train_prob = pipeline.predict_proba(X_train[features])[:, 1]
    val_prob = pipeline.predict_proba(X_val[features])[:, 1]

    # Calculate AUC
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    results.append([full_model_name, train_auc, val_auc])

# Adjust DataFrame to accommodate new results format
results_df = pd.DataFrame(results, columns=['Model', 'Training AUC', 'Validation AUC'])
results_df


,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
5,M6 Logistic Regression,0.693258,0.694652
6,M7 Logistic Regression,0.694248,0.695419
7,M8 Logistic Regression,0.695059,0.696528
8,M9 Logistic Regression,0.695749,0.694356
9,M10 Logistic Regression,0.699248,0.694025


## Lasso Model
---

In [61]:
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import time

for group_name, features in models.items():
    start_time = time.time()  # Start timer

    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("lasso", LassoCV())
    ]
    pipe_lasso = Pipeline(steps)
    pipe_lasso.fit(X_train[features], y_train)

    # Assuming y_train and y_val are binary and you're treating Lasso predictions as continuous scores
    train_scores = pipe_lasso.predict(X_train[features])
    val_scores = pipe_lasso.predict(X_val[features])

    # Convert scores to binary predictions based on a threshold, e.g., the median
    threshold = np.median(train_scores)  # Simple threshold example
    train_pred = np.where(train_scores > threshold, 1, 0)
    val_pred = np.where(val_scores > threshold, 1, 0)

    # Calculate AUC scores
    train_auc = roc_auc_score(y_train, train_pred)
    val_auc = roc_auc_score(y_val, val_pred)

    new_row = pd.DataFrame([[f"{group_name} Lasso", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    end_time = time.time()  # End timer
    print(f"Completed {group_name} in {end_time - start_time:.2f} seconds")

results_df

Completed M1 in 0.36 seconds
Completed M2 in 0.36 seconds
Completed M3 in 0.52 seconds
Completed M4 in 0.52 seconds
Completed M5 in 0.60 seconds
Completed M6 in 0.81 seconds
Completed M7 in 0.78 seconds
Completed M8 in 0.83 seconds
Completed M9 in 1.31 seconds
Completed M10 in 1.86 seconds
Completed M11 in 1.82 seconds


,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
5,M6 Logistic Regression,0.693258,0.694652
6,M7 Logistic Regression,0.694248,0.695419
7,M8 Logistic Regression,0.695059,0.696528
8,M9 Logistic Regression,0.695749,0.694356
9,M10 Logistic Regression,0.699248,0.694025


## Random Forest 
---

In [62]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import time

for group_name, features in models.items():
    start_time = time.time()  # Start timer
    
    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("tree", DecisionTreeClassifier(max_depth=5, random_state=20240407))
    ]
    pipe_tree = Pipeline(steps)

    # Fit the model on training data
    pipe_tree.fit(X_train[features], y_train)

    # Predict probabilities for the positive class
    train_prob = pipe_tree.predict_proba(X_train[features])[:, 1]
    val_prob = pipe_tree.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    new_row = pd.DataFrame([[f"{group_name} Decision Tree MD5", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {group_name} in {end_time - start_time:.2f} seconds")

results_df


Completed M1 in 0.12 seconds
Completed M2 in 0.22 seconds
Completed M3 in 0.40 seconds
Completed M4 in 0.40 seconds
Completed M5 in 0.42 seconds
Completed M6 in 0.64 seconds
Completed M7 in 0.65 seconds
Completed M8 in 0.65 seconds
Completed M9 in 0.73 seconds
Completed M10 in 1.13 seconds
Completed M11 in 1.15 seconds


,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
5,M6 Logistic Regression,0.693258,0.694652
6,M7 Logistic Regression,0.694248,0.695419
7,M8 Logistic Regression,0.695059,0.696528
8,M9 Logistic Regression,0.695749,0.694356
9,M10 Logistic Regression,0.699248,0.694025


In [63]:
for group_name, features in models.items():
    start_time = time.time()  # Start timer
    
    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("tree", DecisionTreeClassifier(max_depth=6, random_state=20240407))
    ]
    pipe_tree = Pipeline(steps)

    # Fit the model on training data
    pipe_tree.fit(X_train[features], y_train)

    # Predict probabilities for the positive class
    train_prob = pipe_tree.predict_proba(X_train[features])[:, 1]
    val_prob = pipe_tree.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    new_row = pd.DataFrame([[f"{group_name} Decision Tree MD6", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {group_name} in {end_time - start_time:.2f} seconds")

results_df

Completed M1 in 0.15 seconds
Completed M2 in 0.26 seconds
Completed M3 in 0.44 seconds
Completed M4 in 0.49 seconds
Completed M5 in 0.47 seconds
Completed M6 in 0.69 seconds
Completed M7 in 0.74 seconds
Completed M8 in 0.76 seconds
Completed M9 in 0.90 seconds
Completed M10 in 1.30 seconds
Completed M11 in 1.41 seconds


,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
5,M6 Logistic Regression,0.693258,0.694652
6,M7 Logistic Regression,0.694248,0.695419
7,M8 Logistic Regression,0.695059,0.696528
8,M9 Logistic Regression,0.695749,0.694356
9,M10 Logistic Regression,0.699248,0.694025


In [64]:
for group_name, features in models.items():
    start_time = time.time()  # Start timer
    
    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("tree", DecisionTreeClassifier(max_depth=7, random_state=20240407))
    ]
    pipe_tree = Pipeline(steps)

    # Fit the model on training data
    pipe_tree.fit(X_train[features], y_train)

    # Predict probabilities for the positive class
    train_prob = pipe_tree.predict_proba(X_train[features])[:, 1]
    val_prob = pipe_tree.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    new_row = pd.DataFrame([[f"{group_name} Decision Tree MD7", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {group_name} in {end_time - start_time:.2f} seconds")

results_df

Completed M1 in 0.16 seconds
Completed M2 in 0.30 seconds
Completed M3 in 0.51 seconds
Completed M4 in 0.55 seconds
Completed M5 in 0.55 seconds
Completed M6 in 0.86 seconds
Completed M7 in 0.88 seconds
Completed M8 in 0.90 seconds
Completed M9 in 1.05 seconds
Completed M10 in 1.56 seconds
Completed M11 in 1.56 seconds


,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
5,M6 Logistic Regression,0.693258,0.694652
6,M7 Logistic Regression,0.694248,0.695419
7,M8 Logistic Regression,0.695059,0.696528
8,M9 Logistic Regression,0.695749,0.694356
9,M10 Logistic Regression,0.699248,0.694025


In [65]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import time

results = []  # Assuming 'results' is your results list

for group_name, features in models.items():
    start_time = time.time()  # Start timer
    
    # Define the steps of the pipeline
    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("tree", DecisionTreeClassifier(random_state=20240407))
    ]
    pipe_tree = Pipeline(steps)
    
    # Define the parameter grid to search over
    param_grid = {
        "tree__max_depth": range(3, 8)  # Adjust the range based on your dataset and prior knowledge
    }
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(pipe_tree, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
    
    # Fit the model on training data
    grid_search.fit(X_train[features], y_train)
    
    # Best model after grid search
    best_model = grid_search.best_estimator_
    
    # Predict probabilities for the positive class with the best model
    train_prob = best_model.predict_proba(X_train[features])[:, 1]
    val_prob = best_model.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    best_depth = best_model.named_steps['tree'].max_depth
    new_row = pd.DataFrame([[f"{group_name} Decision Tree MD{best_depth}", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {group_name} with best max_depth={best_depth} in {end_time - start_time:.2f} seconds")

results_df


Completed M1 with best max_depth=7 in 7.77 seconds
Completed M2 with best max_depth=5 in 1.18 seconds
Completed M3 with best max_depth=4 in 1.58 seconds
Completed M4 with best max_depth=5 in 1.70 seconds
Completed M5 with best max_depth=5 in 1.88 seconds
Completed M6 with best max_depth=4 in 2.64 seconds
Completed M7 with best max_depth=4 in 2.67 seconds
Completed M8 with best max_depth=4 in 2.75 seconds
Completed M9 with best max_depth=4 in 3.11 seconds
Completed M10 with best max_depth=5 in 5.13 seconds
Completed M11 with best max_depth=5 in 5.02 seconds


,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
...,...,...,...
61,M7 Decision Tree MD4,0.688161,0.671863
62,M8 Decision Tree MD4,0.688161,0.671863
63,M9 Decision Tree MD4,0.688161,0.671863
64,M10 Decision Tree MD5,0.699318,0.672435


## Random Forest
---

In [68]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import time

for group_name, features in models.items():
    start_time = time.time()  # Start timer
    
    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("random_forest", RandomForestClassifier(random_state=20240407))
    ]
    pipe_rf = Pipeline(steps)
    
    # Define the parameter grid to search over
    param_grid = {
        "random_forest__max_depth": [None, 3, 5, 7],  # None means no limit on the depth
        "random_forest__n_estimators": [10, 50, 100],  # Number of trees
        "random_forest__min_samples_split": [2, 4]  # Minimum number of samples required to split an internal node
    }
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(pipe_rf, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
    
    # Fit the model on training data
    grid_search.fit(X_train[features], y_train)
    
    # Best model after grid search
    best_model = grid_search.best_estimator_
    
    # Predict probabilities for the positive class with the best model
    train_prob = best_model.predict_proba(X_train[features])[:, 1]
    val_prob = best_model.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    best_params = grid_search.best_params_
    new_row = pd.DataFrame([[f"{group_name} Random Forest", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {group_name} with best parameters {best_params} in {end_time - start_time:.2f} seconds")

results_df


Completed M1 with best parameters {'random_forest__max_depth': 7, 'random_forest__min_samples_split': 2, 'random_forest__n_estimators': 100} in 24.57 seconds
Completed M2 with best parameters {'random_forest__max_depth': 7, 'random_forest__min_samples_split': 4, 'random_forest__n_estimators': 100} in 32.80 seconds
Completed M3 with best parameters {'random_forest__max_depth': 7, 'random_forest__min_samples_split': 2, 'random_forest__n_estimators': 100} in 42.97 seconds
Completed M4 with best parameters {'random_forest__max_depth': 7, 'random_forest__min_samples_split': 2, 'random_forest__n_estimators': 100} in 52.75 seconds
Completed M5 with best parameters {'random_forest__max_depth': 7, 'random_forest__min_samples_split': 2, 'random_forest__n_estimators': 100} in 45.23 seconds
Completed M6 with best parameters {'random_forest__max_depth': 7, 'random_forest__min_samples_split': 4, 'random_forest__n_estimators': 100} in 58.26 seconds
Completed M7 with best parameters {'random_forest__m

,Model,Training AUC,Validation AUC
0,M1 Logistic Regression,0.548108,0.555135
1,M2 Logistic Regression,0.624687,0.627810
2,M3 Logistic Regression,0.682657,0.686424
3,M4 Logistic Regression,0.686342,0.688129
4,M5 Logistic Regression,0.687929,0.685065
...,...,...,...
72,M7 Random Forest,0.791677,0.714396
73,M8 Random Forest,0.794655,0.715435
74,M9 Random Forest,0.793159,0.713248
75,M10 Random Forest,0.793052,0.715690


In [70]:
results_df.tail(11)

,Model,Training AUC,Validation AUC
66,M1 Random Forest,0.692130,0.603921
67,M2 Random Forest,0.733817,0.656886
68,M3 Random Forest,0.774158,0.708519
69,M4 Random Forest,0.782287,0.711795
70,M5 Random Forest,0.783116,0.710803
71,M6 Random Forest,0.793772,0.714494
72,M7 Random Forest,0.791677,0.714396
73,M8 Random Forest,0.794655,0.715435
74,M9 Random Forest,0.793159,0.713248
75,M10 Random Forest,0.793052,0.715690


## Gradient Boosted Random Forest
---

In [71]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import time



for group_name, features in models.items():
    start_time = time.time()  # Timer start
    
    steps = [
        ("scale_features", ColumnTransformer([("scale_numeric_features", MinMaxScaler(), features)], remainder='drop')),
        ("xgb", xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
    ]
    pipe_xgb = Pipeline(steps)
    
    # Define the parameter grid
    param_grid = {
        "xgb__n_estimators": [100, 200],  # Number of trees
        "xgb__max_depth": [3, 5, 7],  # Depth of trees
        "xgb__learning_rate": [0.01, 0.1]  # Step size shrinkage used in update to prevents overfitting
    }
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(pipe_xgb, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
    
    # Fit the model on the training data
    grid_search.fit(X_train[features], y_train)
    
    # Best model after grid search
    best_model = grid_search.best_estimator_
    
    # Predict probabilities for the positive class
    train_prob = best_model.predict_proba(X_train[features])[:, 1]
    val_prob = best_model.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    best_params = grid_search.best_params_
    new_row = pd.DataFrame([[f"{group_name} XGBoost", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {group_name} with best parameters {best_params} in {end_time - start_time:.2f} seconds")

results_df.tail(11)


Completed M1 with best parameters {'xgb__learning_rate': 0.01, 'xgb__max_depth': 5, 'xgb__n_estimators': 200} in 4.42 seconds
Completed M2 with best parameters {'xgb__learning_rate': 0.01, 'xgb__max_depth': 5, 'xgb__n_estimators': 200} in 6.50 seconds
Completed M3 with best parameters {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100} in 9.71 seconds
Completed M4 with best parameters {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100} in 11.33 seconds
Completed M5 with best parameters {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100} in 11.23 seconds
Completed M6 with best parameters {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100} in 17.02 seconds
Completed M7 with best parameters {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100} in 17.90 seconds
Completed M8 with best parameters {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100} in 19.48 s

,Model,Training AUC,Validation AUC
77,M1 XGBoost,0.665216,0.596593
78,M2 XGBoost,0.719108,0.659764
79,M3 XGBoost,0.747470,0.712439
80,M4 XGBoost,0.759199,0.716104
81,M5 XGBoost,0.759799,0.717293
82,M6 XGBoost,0.769568,0.720461
83,M7 XGBoost,0.771006,0.720639
84,M8 XGBoost,0.771006,0.720639
85,M9 XGBoost,0.770794,0.719920
86,M10 XGBoost,0.772891,0.718886


## Explainable Boosting Machine
---

In [77]:
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import time

for group_name, features in models.items():
    start_time = time.time()  # Timer start

    # Adjusted EBM pipeline without SimpleImputer for numerical data
    ebm = ExplainableBoostingClassifier(random_state=20240325)

    ebm.fit(X_train[features], y_train)

    # Predict probabilities for the positive class
    train_prob = ebm.predict_proba(X_train[features])[:, 1]
    val_prob = ebm.predict_proba(X_val[features])[:, 1]

    # Calculate AUC scores using the probabilities
    train_auc = roc_auc_score(y_train, train_prob)
    val_auc = roc_auc_score(y_val, val_prob)

    # Append results
    new_row = pd.DataFrame([[f"{group_name} EBM", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    end_time = time.time()  # End timer
    print(f"Completed {group_name} in {end_time - start_time:.2f} seconds")

results_df.tail(11)


Completed M1 in 6.94 seconds
Completed M2 in 5.66 seconds
Completed M3 in 8.72 seconds
Completed M4 in 8.40 seconds
Completed M5 in 10.49 seconds
Completed M6 in 16.92 seconds
Completed M7 in 18.97 seconds
Completed M8 in 20.23 seconds
Completed M9 in 21.68 seconds
Completed M10 in 30.66 seconds
Completed M11 in 32.78 seconds


,Model,Training AUC,Validation AUC
88,M1 EBM,0.628037,0.599450
89,M2 EBM,0.687042,0.658695
90,M3 EBM,0.727854,0.710408
91,M4 EBM,0.743061,0.718112
92,M5 EBM,0.747173,0.718324
93,M6 EBM,0.763240,0.724013
94,M7 EBM,0.765587,0.726944
95,M8 EBM,0.768218,0.725339
96,M9 EBM,0.764281,0.727483
97,M10 EBM,0.766671,0.726923


## Neural Network Models
---

In [82]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


for model_name, features in models.items():
    start_time = time.time()  # Timer start
    # Prepare the data
    X_temp = X[features]
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y, test_size=0.2, random_state=20240325)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Define the model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[AUC(name='auc')])
    
    model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=0,
              validation_data=(X_val_scaled, y_val),
              callbacks=[EarlyStopping(monitor='val_auc', patience=3, restore_best_weights=True, mode='max')])

    # Evaluate the model on training data
    _, train_auc = model.evaluate(X_train_scaled, y_train, verbose=0)

    # Evaluate the model on validation data
    _, val_auc = model.evaluate(X_val_scaled, y_val, verbose=0)
    
    # Append the results
    new_row = pd.DataFrame([[f"{model_name} NN Simple", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()  # End timer
    print(f"Completed {model_name} in {end_time - start_time:.2f} seconds")

results_df.tail(11)

Completed M1 in 8.68 seconds
Completed M2 in 7.15 seconds
Completed M3 in 11.08 seconds
Completed M4 in 14.26 seconds
Completed M5 in 10.38 seconds
Completed M6 in 8.69 seconds
Completed M7 in 6.29 seconds
Completed M8 in 6.06 seconds
Completed M9 in 8.85 seconds
Completed M10 in 8.53 seconds
Completed M11 in 5.07 seconds


,Model,Training AUC,Validation AUC
100,M1,0.586929,0.587971
101,M2,0.659303,0.645004
102,M3,0.718505,0.701441
103,M4,0.734196,0.708390
104,M5,0.737095,0.703616
105,M6,0.754547,0.710161
106,M7,0.739022,0.707021
107,M8,0.730697,0.700787
108,M9,0.759777,0.704857
109,M10,0.750669,0.695664


In [83]:
from tensorflow.keras.layers import Dropout, BatchNormalization

for model_name, features in models.items():
    start_time = time.time()
    X_temp = X[features]
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y, test_size=0.2, random_state=20240325)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=[AUC(name='auc')])
    
    model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=0,
              validation_data=(X_val_scaled, y_val),
              callbacks=[EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True, mode='max')])

    _, train_auc = model.evaluate(X_train_scaled, y_train, verbose=0)
    _, val_auc = model.evaluate(X_val_scaled, y_val, verbose=0)
    
    new_row = pd.DataFrame([[f"{model_name} NN Complex", train_auc, val_auc]], columns=['Model', 'Training AUC', 'Validation AUC'])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    end_time = time.time()
    print(f"Completed {model_name} in {end_time - start_time:.2f} seconds")

results_df.tail(11)


Completed M1 in 16.08 seconds
Completed M2 in 29.03 seconds
Completed M3 in 41.43 seconds
Completed M4 in 34.68 seconds
Completed M5 in 50.32 seconds
Completed M6 in 31.34 seconds
Completed M7 in 32.70 seconds
Completed M8 in 30.41 seconds
Completed M9 in 32.38 seconds
Completed M10 in 28.50 seconds
Completed M11 in 38.59 seconds


,Model,Training AUC,Validation AUC
111,M1 NN Complex,0.571327,0.579950
112,M2 NN Complex,0.666053,0.648391
113,M3 NN Complex,0.721719,0.705432
114,M4 NN Complex,0.729202,0.711933
115,M5 NN Complex,0.744800,0.714671
116,M6 NN Complex,0.744320,0.720790
117,M7 NN Complex,0.752126,0.713864
118,M8 NN Complex,0.745184,0.715554
119,M9 NN Complex,0.749792,0.714372
120,M10 NN Complex,0.732641,0.711651


# Hypertuning
---

## Feature Importance

# Model Selection
---

In [84]:
# Assuming 'results_df' contains your model results with 'Model', 'Training AUC', and 'Validation AUC' columns

# Add a 'Difference AUC' column to measure overfitting
results_df['Difference AUC'] = abs(results_df['Training AUC'] - results_df['Validation AUC'])

# Add a 'Complexity' column based on the model name. Assuming 'M1' is simpler than 'M11'.
# This extracts the numeric part of the model name for sorting.
results_df['Complexity'] = results_df['Model'].apply(lambda x: int(x.split()[0][1:]))

# Sort by Validation AUC (desc), then by Difference AUC (asc), then by Complexity (asc)
sorted_results_df = results_df.sort_values(by=['Validation AUC', 'Difference AUC', 'Complexity'], ascending=[False, True, True])

# Get the top 10 models
top_10_models = sorted_results_df.head(10)
top_10_models


,Model,Training AUC,Validation AUC,Difference AUC,Complexity
96,M9 EBM,0.764281,0.727483,0.036797,9
94,M7 EBM,0.765587,0.726944,0.038643,7
97,M10 EBM,0.766671,0.726923,0.039748,10
98,M11 EBM,0.766508,0.726586,0.039922,11
95,M8 EBM,0.768218,0.725339,0.042880,8
93,M6 EBM,0.763240,0.724013,0.039227,6
116,M6 NN Complex,0.744320,0.720790,0.023530,6
83,M7 XGBoost,0.771006,0.720639,0.050367,7
84,M8 XGBoost,0.771006,0.720639,0.050367,8
82,M6 XGBoost,0.769568,0.720461,0.049106,6


# Test Set Prediction
---

In [88]:
models['M9']

['n_tokens_title',
 'n_tokens_content',
 'n_unique_tokens',
 'n_non_stop_words',
 'n_non_stop_unique_tokens',
 'average_token_length',
 'num_keywords',
 'num_hrefs',
 'num_self_hrefs',
 'num_imgs',
 'num_videos',
 'global_subjectivity',
 'global_sentiment_polarity',
 'global_rate_positive_words',
 'global_rate_negative_words',
 'kw_min_min',
 'kw_max_min',
 'kw_avg_min',
 'kw_min_max',
 'kw_max_max',
 'kw_avg_max',
 'kw_min_avg',
 'kw_max_avg',
 'kw_avg_avg',
 'self_reference_min_shares',
 'self_reference_max_shares',
 'self_reference_avg_sharess',
 'weekday_is_monday',
 'weekday_is_tuesday',
 'weekday_is_wednesday',
 'weekday_is_thursday',
 'weekday_is_friday',
 'weekday_is_saturday',
 'weekday_is_sunday',
 'is_weekend',
 'data_channel_is_lifestyle',
 'data_channel_is_entertainment',
 'data_channel_is_bus',
 'data_channel_is_socmed',
 'data_channel_is_tech',
 'data_channel_is_world',
 'LDA_00',
 'LDA_01',
 'LDA_02',
 'LDA_03',
 'LDA_04',
 'rate_positive_words',
 'rate_negative_words',

In [90]:
test_data.columns

Index(['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'glob

In [96]:
from interpret.glassbox import ExplainableBoostingClassifier

# Assuming "M9" features and model setup
m9_features = models['M9']

# Training the "M9" EBM model
ebm_m9 = ExplainableBoostingClassifier(random_state=20240325)
ebm_m9.fit(X_train[m9_features], y_train)

# Assuming test_data is prepared and has the same features used for training "M9"
X_test_m9 = test_data[m9_features]

# Predicting with "M9" model
test_data['score'] = ebm_m9.predict_proba(X_test_m9)[:, 1]

# Saving the required predictions
test_data[['article_id', 'score']].to_csv('m9_predictions.csv', index=False)
